In [3]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from jupyter_dash import JupyterDash
from dash import Dash, html, dcc, Input, Output, callback , State, ctx, dash_table
import plotly.express as px
import pandas as pd

import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3
import datetime

app = JupyterDash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
global index

def table(ticker):
    con = sqlite3.connect("stock.sqlite")
    cur = con.cursor()
    df = pd.read_sql("SELECT * FROM stock_financial where Ticker = '"+ticker+"';",con)
    column = df['Year'].values.tolist()
    df = df.transpose()
    df.columns = column
    df = df.drop("Ticker")
    df = df.drop("Year")
    head = df.index.values.tolist()
    df.insert(0, "Financial Information", head, True)
    return df

def plot(ticker, interval):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    query1 = "select * from stock_table_hr where `ticker` == '%s' and datetime > '2023-01-01' and datetime < '2023-01-06'" % ticker
    query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-05-01' and datetime < '2023-01-06'" % ticker
    query3 = "select * from stock_table_mo where `ticker` == '%s' and datetime > '2022-01-05' and datetime < '2023-01-06'" % ticker
    Hr = pd.read_sql(query1,conn)
    Day = pd.read_sql(query2,conn)
    Mo = pd.read_sql(query3,conn)
    #declare figure
    fig = go.Figure()
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                vertical_spacing=0.1, subplot_titles=('OHLC', 'Volume'), 
                row_width=[0.2, 0.7])

    #Candlestick

    # fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])
    if interval == "Hour":
        Hr['time'] = pd.to_datetime(Hr['Datetime'])
        Hr['MA50'] = Hr['Close'].rolling(window = 50 , min_periods = 0).mean()
        Hr['MA200'] = Hr['Close'].rolling(window = 200 , min_periods = 0).mean()
        Hr.set_index(Hr['time'],inplace = True)
        fig.add_trace(go.Candlestick(x=Hr.Datetime,
                        open=Hr.Open,
                        high=Hr.High,
                        low=Hr.Low,
                        close=Hr.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=Hr.Datetime,y=Hr['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=Hr.Datetime,y=Hr['MA200']),row=1, col=1)
        fig.add_trace(go.Bar(x=Hr.Datetime, y=Hr.Volume,showlegend=False), row=2, col=1)

        df_resample_hr = Hr.resample('H').max()
        merged_index_hr  = Hr.index.append(df_resample_hr.index)
        timegap_hr = merged_index_hr[~merged_index_hr.duplicated(keep = False)]

        fig.update_xaxes(rangebreaks=[dict(values=timegap_hr, dvalue=3600000)])
        
    elif interval == "Day":
        Day['time'] = pd.to_datetime(Day['Datetime'])
        Day['MA50'] = Day['Close'].rolling(window = 50 , min_periods = 0).mean()
        Day['MA200'] = Day['Close'].rolling(window = 200 , min_periods = 0).mean()
        Day.set_index(Day['time'],inplace = True)
        fig.add_trace(go.Candlestick(x=Day.Datetime,
                        open=Day.Open,
                        high=Day.High,
                        low=Day.Low,
                        close=Day.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=Day.Datetime,y=Day['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=Day.Datetime,y=Day['MA200']),row=1, col=1)
        fig.add_trace(go.Bar(x=Day.Datetime, y=Day.Volume,showlegend=False), row=2, col=1)

    
        df_resample_day = Day.resample('D').max()
        merged_index_day  = Day.index.append(df_resample_day.index)
        timegap_day = merged_index_day[~merged_index_day.duplicated(keep = False)]

        fig.update_xaxes(rangebreaks=[dict(values=timegap_day)])

    elif interval == "Month":
        Mo['time'] = pd.to_datetime(Mo['Datetime'])
        Mo['MA50'] = Mo['Close'].rolling(window = 50 , min_periods = 0).mean()
        Mo['MA200'] = Mo['Close'].rolling(window = 200 , min_periods = 0).mean()
        Mo.set_index(Mo['time'],inplace = True)
        fig.add_trace(go.Candlestick(x=Mo.Datetime,
                        open=Mo.Open,
                        high=Mo.High,
                        low=Mo.Low,
                        close=Mo.Close, 
                        name = ticker,),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA50',x=Mo.Datetime,y=Mo['MA50']),row=1, col=1)
        fig.add_trace(go.Scatter(name = 'MA200',x=Mo.Datetime,y=Mo['MA200']),row=1, col=1)
        fig.add_trace(go.Bar(x=Mo.Datetime, y=Mo.Volume,showlegend=False), row=2, col=1)


    fig.update_yaxes(rangemode="nonnegative")
    fig.update_xaxes(rangemode="nonnegative")
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(hovermode = "x")
    fig.update(layout_xaxis_rangeslider_visible=False)
    return fig

def getLastDate(period,ticker):
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    # Query last element of stock in database
    if period == 'Hour':
        query = "SELECT * FROM stock_table_hr WHERE `ticker` = '%s'" % ticker
    elif period == 'Day':
        query = "SELECT * FROM stock_table_d WHERE `ticker` = '%s'" % ticker
    elif period == 'Mount':
        query = "SELECT * FROM stock_table_mo WHERE `ticker` = '%s'" % ticker
    else:
        return False
    r_df = pd.read_sql(query, conn)
    # Cut data to get only datatime
    last = r_df.tail(1).Datetime.to_string().split()
    LastDate = last[1].split()[0].split('-')
    cur.close()
    return LastDate,r_df

def getDiffDay(period,ticker):
    LastDate,r_df = getLastDate(period,ticker)
    if LastDate == False:
        return False
    # Get datetime for now
    x = datetime.datetime.now()
    count = 0
    DayM = 0
    DayMo365 = {'1':31,'2':28,'3':31,'4':30,'5':31,'6':30,'7':31,'8':31,'9':30,'10':31,'11':30,'12':31}
    DiffMo = int(x.month) - int(LastDate[1])
    DiffYe = int(x.year) - int(LastDate[0])
    # Get differend day for dowload stock
    if DiffYe == 0:
        if DiffMo == 0:
            DiffDay = int(x.day) - int(LastDate[2])
            if DiffDay != 0:
                pass
        elif DiffMo != 0 :
            for u in range(DiffMo):
                DayM = DayM + DayMo365[str(int(LastDate[1])+count)]
                count += 1
            DiffDay = DayM - int(LastDate[2]) + int(x.day)
    elif DiffYe != 0:
        dayly = 0
        dayn = 0
        for j in range(1,int(LastDate[1])):
            dayly = dayly + DayMo365[str(j)]
        for i in range(1,int(x.month)):
            dayn = dayn + DayMo365[str(i)]
        DiffDay = (365*DiffYe) - dayly + dayn - int(LastDate[2]) + int(x.day)   
    DiffDay = str(DiffDay) + 'd'
    return DiffDay
    
    
def update(period,ticker):
    LastDate,r_df = getLastDate(period,ticker)
    DiffDay = getDiffDay(period,ticker)
    if DiffDay == False:
        return False
    down = 0
    count = 0
    conn = sqlite3.connect("stock.sqlite")
    # Select period to download
    if period == 'Hour':
        data = yf.download(tickers=ticker, period=DiffDay, interval='1h')
    elif period == 'Day':
        data = yf.download(tickers=ticker, period=DiffDay, interval='1d')
    elif period == 'Mount':
        data = yf.download(tickers=ticker, period=DiffDay, interval='1mo')
    # Get number of extra stock
    for i in data.index.day:
        if data.index.year[count] == int(LastDate[0]):
            if data.index.month[count] == int(LastDate[1]):
                if i == int(LastDate[2])+1:
                    break
        count += 1
    ok = r_df.tail(1).Datetime.to_string().split()[2]
    if ok == '10:00:00':
        down = 5
    elif ok == '11:00:00':
        down = 4
    elif ok == '12:00:00':
        down = 3
    elif ok == '14:00:00':
        down = 2
    elif ok == '15:00:00':
        down = 1
    elif ok == '16:00:00':
        down = 0
    # Cut extra stock off
    count = count - down
    data['ticker'] = ticker
    data = data.iloc[count:,:]
    # Save to sqlite
    # Select period to download
    # if period == 'Hour':
    #     data.to_sql('stock_table_hr',con=conn,if_exists='append',index=True)
    # elif period == 'Day':
    #     data.to_sql('stock_table_d',con=conn,if_exists='append',index=True)
    # elif period == 'Mount':
    #     data.to_sql('stock_table_mo',con=conn,if_exists='append',index=True)
    return data
        
def stock_name():
    conn = sqlite3.connect("stock.sqlite")
    cur = conn.cursor()
    query = "select distinct Ticker from stock_table_hr"
    r_df = pd.read_sql(query,conn)
    list_db = r_df['Ticker'].values.tolist()
    return list_db

def download_ticker(period,ticker):
        conn = sqlite3.connect("stock.sqlite")
        cur = conn.cursor()
        try:
            # Select period to download
            if period == 'Hour':
                data = yf.download(tickers=ticker, period='2y', interval='1h')
            elif period == 'Day':
                data = yf.download(tickers=ticker, period='max', interval='1d')
            elif period == 'Mount':
                data = yf.download(tickers=ticker, period='max', interval='1mo')
            else:
                data = []
            # Save to sqlite3
            # data.to_sql('stock_table',con=conn,if_exists='append',index=True)
            # return data to ploting graph
            if data.empty:
                raise ValueError
            return data
        except:
            return []

######DASH######
app.layout = html.Div(children=[
    html.Div([
        html.H1(children='Stock Radar',
                style={
                    'textAlign': 'center',
                    }
        ),
        html.Div([
            "Stock : ",
            dcc.Input(id='ticker-input', value='', type='text'),
            html.Button(id='submit-button', n_clicks=0, children='Submit'),
        ],
            style={
                'margin-bottom': 20,
            }
        ),
        dcc.Dropdown(['Hour', 'Day', 'Month'], 
            searchable=False, 
            id='stock-dropdown',
            style={
                'margin-bottom': 20,
            }
        ),
        html.Div(id='output-div'),
        html.Div(id="disp-ticker"),
        html.Button('Show Table', id='show-table-button'),
        ],
        style={
            'margin-top' : 50,
            'margin-right': 100,
            'margin-left': 100
        }
    ),
    html.Div([
        dcc.Graph(
            id='graph',
            figure=plot('','Hour')
        )],
        style={
                'margin-right': 20,
                'margin-left': 20
        }
    ),

    dcc.ConfirmDialog(
        id='popup',
        message='Not found that tickers',
        displayed=False,
    ),
    dcc.ConfirmDialog(
        id='popup_dl',
        message='Downloaded',
        displayed=False,
    ),
    dcc.ConfirmDialog(
        id='popup_fa',
        message='False to download for this ticker',
        displayed=False,
    ),
    html.Div(id='table-container'),


])

@app.callback([Output('graph','style'),Output('popup', 'displayed')],
              [Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')])

def update_popup_and_graph(n_clicks, value):
    list_db = stock_name()
    value = value.upper()
    if n_clicks:
        if value.strip() == '' or (value.strip() not in list_db):
            return plot('','Hour'),True
        else:
            return plot(value, 'Hour'),False
    return plot('', 'Hour'),False

#return graph from ticker input
@app.callback(Output('graph','figure'),
              [Input('stock-dropdown', 'value'), 
              Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')],)
def update_graph(stock_dropdown, submit_clicks, ticker_input):
    print(ticker_input)
    if stock_dropdown != None:
        return dropdown_output(stock_dropdown, index)
    elif submit_clicks is not None:
        index = ticker_input
        return update_output(ticker_input)   

def update_output(ticker_input):
    list_db = stock_name()
    value = ticker_input.upper()
    if value.strip() == '' or (value.strip() not in list_db):
            fig = plot('','Hour')
            return fig
    else:
        for i in ['Hour','Day','Mount']:
            update(i,value)
        value = value.upper()
        fig = plot(value,'Hour')
        return fig

def dropdown_output(stock_dropdown ,ticker_input):
    value = str(ticker_input).upper()
    fig = plot(value, stock_dropdown)
    return fig


#
@app.callback(
    [Output("popup_dl", "displayed"),Output("popup_fa", "displayed")],
    [Input("popup", "submit_n_clicks")],
    [State('ticker-input', 'value')]
)
def execute_function(submit_n_clicks ,value):
    data = download_ticker('Hour',value)
    if submit_n_clicks and data != []:
        return True,False
    elif submit_n_clicks and data == []:
        return False,True
    return False,False

#reset value dropdown
@app.callback(Output('stock-dropdown', 'value'),
              Output("disp-ticker", "children"),
              [Input('submit-button', 'n_clicks')],
              [State('ticker-input', 'value')]
             )
def reset_dropdown(n_clicks, value):
    return None, value.upper()

#show table when press button
@app.callback(
    Output('table-container', 'children'),
    [Input('show-table-button', 'n_clicks'),
    Input('submit-button', 'n_clicks')]
)
def show_table(n_clicks1, n_clicks2):
    df = table(index)
    if "submit-button" == ctx.triggered_id:
        return []

    elif "show-table-button" == ctx.triggered_id: 
        return dash_table.DataTable(df.to_dict('records'), [{"name": i, "id": i} for i in df.columns],
            id='table-financial',
            style_cell={
                'whiteSpace': 'normal',
                'height': 'auto'},
                fill_width=False
        )
    
    




app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


[*********************100%***********************]  0 of 0 completed

ACE.BK
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.I("Try typing in input 1 & 2, and observe how debounce is impacting the callbacks. Press Enter and/or Tab key in Input 2 to cancel the delay"),
        html.Br(),
        dcc.Input(id="input1", type="text", placeholder="", style={'marginRight':'10px'}),
        dcc.Input(id="input2", type="text", placeholder="", debounce=True),
        html.Div(id="output"),
    ]
)


@app.callback(
    Output("output", "children"),
    Input("input1", "value"),
    Input("input2", "value"),
)
def update_output(input1, input2):
    return u'Input 1 {} and Input 2 {}'.format(input1, input2)



app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [50]:
con = sqlite3.connect("stock.sqlite")
cur = con.cursor()
df = pd.read_sql("SELECT * FROM stock_financial where Ticker = 'AAV.BK';",con)
column = df['Year'].values.tolist()
df = df.transpose()
df.columns = column
df = df.drop("Ticker")
df = df.drop("Year")
head = df.index.values.tolist()
df.insert(0, "Financial Information", head, True)
print(df)



                                                                                Financial Information  \
Financial Period                                                                     Financial Period   
Total Assets                                                                             Total Assets   
Total Liabilities                                                                   Total Liabilities   
Equity                                                                                         Equity   
Issued and paid-up share capital                                     Issued and paid-up share capital   
Gross Profit                                                                             Gross Profit   
Operating Revenue                                                                   Operating Revenue   
Total Costs and Expenses                                                     Total Costs and Expenses   
Profit Before Finance Costs and Income Tax Expe...  Pro

In [36]:

from jupyter_dash import JupyterDash
from dash import Dash, html, Input, Output, ctx

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Button('Button 1', id='btn-nclicks-1', n_clicks=0),
    html.Button('Button 2', id='btn-nclicks-2', n_clicks=0),
    html.Button('Button 3', id='btn-nclicks-3', n_clicks=0),
    html.Div(id='container-button-timestamp')
])

@app.callback(
    Output('container-button-timestamp', 'children'),
    Input('btn-nclicks-1', 'n_clicks'),
    Input('btn-nclicks-2', 'n_clicks'),
    Input('btn-nclicks-3', 'n_clicks')
)
def displayClick(btn1, btn2, btn3):
    print(ctx.triggered_id)
    msg = "None of the buttons have been clicked yet"
    if "btn-nclicks-1" == ctx.triggered_id:
        msg = "Button 1 was most recently clicked"
    elif "btn-nclicks-2" == ctx.triggered_id:
        msg = "Button 2 was most recently clicked"
    elif "btn-nclicks-3" == ctx.triggered_id:
        msg = "Button 3 was most recently clicked"
    return html.Div(msg)

if __name__ == '__main__':
    app.run_server(debug=True)



None
Dash app running on http://127.0.0.1:8050/


btn-nclicks-1
btn-nclicks-1
btn-nclicks-2
btn-nclicks-3
